In [1]:
from mcli import predict
import pandas as pd
import json
import re

In [2]:
## convert data label into list of list
# test_data = pd.read_csv("/Users/rui.chen@avalara.com/Downloads/test.tsv", sep="\t")
# test_data["output"] = test_data["output"].map(json.loads)
# test_data["output"] = test_data["output"].map(lambda x: [list(obj.values()) for obj in x])
# test_data.to_csv("test.tsv", sep="\t")

In [3]:
test_data = pd.read_csv("test.tsv", sep="\t")

finetuned_host = "https://mpt-30b-raw-sut-finetuned-gytdzl.inf.hosted-on.mosaicml.hosting"
test_data

,Unnamed: 0,document_url,input,output
0,0,2020_Apr_George-740d04.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['George TBD', 'Local Sales Tax Rate', '0.019..."
1,1,2020_Apr_GigHarbor-92d49d.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Gig Harbor', 'Local Sales Tax Rate', '0.022..."
2,2,2020_Apr_GraysHarbor-bf4ccd.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
3,3,2020_Apr_GraysHarborLodging-d073f7.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Combined Sales Tax Rate'..."
4,4,2020_Apr_Yakima-561d81.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
...,...,...,...,...
95,95,local-tax-changes-10-01-2021-dd4828.pdf,"\n,Local Tax Changes\n,Effective October 1, 20...","[['Hankinson', 'City sales, use, and gross rec..."
96,96,local-tax-changes-7-1-2020-b76cfe.pdf,"Effective July 1, 2020. \nThe following local ...","[['Beulah', 'city sales, use, and alcohol gros..."
97,97,tb-01-21-7a5d19.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[['Orderville (Kane County)', 'Transient Room ..."
98,98,tb-02-22-a5b2d2.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[['Eagle Mountain (Utah County)', 'Transient R..."


In [4]:
failed_indices = []
predictions_finetuned = []
for i, row in test_data.iterrows():
    try:
        i_prediction = predict(finetuned_host, 
                                    {
                                        "inputs": [row["input"]], 
                                        "temperature": 0.,
                                        "max_length": 8192,
                                    }, 
                                    timeout=600)
        predictions_finetuned.append(i_prediction["output"][0])
        # print(i_prediction["output"][0])
        print(i)
    except:
        print(f"{i} failed")
        failed_indices.append("[[]]")



In [ ]:
predictions_finetuned[78]

"\n,Kanabec\xa0County\xa00.5\xa0Percent\xa0Transit\xa0Sales\xa0And\xa0Use\xa0Tax\n,Starting January 1, 2020, Kanabec County will have a 0.5 percent transit sales and use tax. The Minnesota\n,Department of Revenue will administer this tax. Revenues will fund Kanabec County Transportation\n,\n,his 0.5 percent transit sales tax applies to retail sales made into Kanabec County. The use tax applies to\n,taxable items used in the County if the local sales tax was not paid.\n,For more information, see Fact Sheet 164, Local Sales and Use Taxes.\n,Who\xa0must\xa0register\n,All retailers who have a taxable presence (nexus) in Kanabec County even if an out-of-state retailer or\n,marketplace and are registered for Minnesota sales tax. This includes all sellers outside Kanabec County\n,if they have a taxable presence such as:\nHave an office, distribution, sales, sample or warehouse locations, or other place of business in,\uf0b7\n. \n,Kanabec\xa0County\xa00.5\xa0Percent\xa0Transit\xa0Sales\xa0And\

In [ ]:
def get_labels(i, text):
    # Search for content including the [[...]]
    # match = re.search(r'(\[\[.*?\]\])', text)
    
    # if match:
    #     content_with_brackets = match.group(1)
    #     return content_with_brackets
    # else:
    #     print("No labels found!", i)
    #     return None

    res = "[[" + text.split("[[")[-1]
    if res.endswith("]]"):
        return res
    else:
        return res + "']]"

    
predictions = [get_labels(i, item) for i, item in enumerate(predictions_finetuned)]

In [ ]:
test_data["prediction"] = predictions
test_data

,Unnamed: 0,document_url,input,output,prediction
0,0,2020_Apr_George-740d04.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['George TBD', 'Local Sales Tax Rate', '0.019...","[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
1,1,2020_Apr_GigHarbor-92d49d.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Gig Harbor', 'Local Sales Tax Rate', '0.022...","[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
2,2,2020_Apr_GraysHarbor-bf4ccd.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Local Sales Tax Rate', '...","[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
3,3,2020_Apr_GraysHarborLodging-d073f7.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Combined Sales Tax Rate'...","[['Unincorp. Areas', 'Lodging Sales Tax Rate',..."
4,4,2020_Apr_Yakima-561d81.pdf,WASHINGTON STATE DEPARTMENT OF REVENUE. LOCAL ...,"[['Unincorp. Areas', 'Local Sales Tax Rate', '...","[['Unincorp. Areas', 'Local Sales Tax Rate', '..."
...,...,...,...,...,...
95,95,local-tax-changes-10-01-2021-dd4828.pdf,"\n,Local Tax Changes\n,Effective October 1, 20...","[['Hankinson', 'City sales, use, and gross rec...","[['Hankinson', 'city sales, use, and gross rec..."
96,96,local-tax-changes-7-1-2020-b76cfe.pdf,"Effective July 1, 2020. \nThe following local ...","[['Beulah', 'city sales, use, and alcohol gros...","[['Beulah', 'July 1, 2020', 'nan', '2', 'city ..."
97,97,tb-01-21-7a5d19.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[['Orderville (Kane County)', 'Transient Room ...","[['Orderville (Kane County)', 'Transient Room ..."
98,98,tb-02-22-a5b2d2.pdf,"\n,Changes in Utah laws or Tax\n,Commission ru...","[['Eagle Mountain (Utah County)', 'Transient R...","[['Eagle Mountain (Utah County)', 'Transient R..."


In [ ]:
test_data.to_csv("mpt30b_raw_sut_predictions.tsv", sep="\t")

In [4]:
predict(finetuned_host, 
                                    {
                                        "inputs": ["Hello"], 
                                        "temperature": 0.,
                                        "max_length": 8192,
                                    }, 
                                    timeout=600)

{'outputs': ["Hello.[['nan', 'nan', 'nan', 'nan', 'nan']]"],
 'output': ["Hello.[['nan', 'nan', 'nan', 'nan', 'nan']]"]}